<a href="https://colab.research.google.com/github/Shehab89/Project1/blob/main/Serendipity_RecSys_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1:
# Enhancing Content Discovery in Recommender Systems: A Serendipity-Based Approach - Proposal
###Supervisor: Dr. Erik Hekman
###Shebab Al-MASRI 4338707


###1- Imports and Setup:
The necessary libraries

In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import linear_kernel

### 2- Data Loading:

Several BBC datasets ( documentaries, history, science, and arts) are loaded into pandas DataFrames using pickle to deserialize the data.

In [3]:
# Replace 'your_file.pkl' with the path to your .pkl file
file_path = '/content/drive/MyDrive/PPM/BBC/documentaries.pkl'

# Open the file in binary read mode
with open(file_path, 'rb') as file:
    # Load the content of the file into a Python object
    bbc_documentries = pickle.load(file)

bbc_documentries

,category,title,tags,description,image,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large
0,Documentaries,Fight for First: Excel Esports - Series 1: 1. ...,"BBC, iPlayer, TV, Fight for First: Excel Espor...",Excel Esports owner Kieran is looking for a fr...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,40 mins,2373,10 Jan 2021,Excel Esports owner Kieran is looking for a fr...,Sports documentary series following British un...,Excel is an esports team founded by brothers K...
1,Documentaries,Lady Boss: The Jackie Collins Story,"BBC, iPlayer, TV, Lady Boss: The Jackie Collin...",An immersive journey through the trailblazing ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,91 mins,5467,15 Oct 2021,An immersive journey through the trailblazing ...,An immersive journey through the trailblazing ...,An immersive journey through the trailblazing ...
2,Documentaries,"The Iraq War - 3. It's Hell, Mr President","BBC, iPlayer, TV, The Iraq War, 3. Its Hell, M...",The final episode tells how the war ended and ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3551,9pm 12 Jun 2013,The final episode tells how the war ended and ...,The biggest players of the war in Iraq reveal ...,The last episode in this three-part series bri...
3,Documentaries,The 80s - Music’s Greatest Decade? - Series 1:...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3534,8:55pm 23 Oct 2021,Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...
4,Documentaries,Sacred Rivers with Simon Reeve - 1. The Nile,"BBC, iPlayer, TV, Sacred Rivers with Simon Ree...",Simon Reeve travels from source to sea along t...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3547,9pm 5 Oct 2014,Simon Reeve travels from source to sea along t...,Simon Reeve explores the influence three river...,Adventurer Simon Reeve travels from source to ...
...,...,...,...,...,...,...,...,...,...,...,...
2774,Documentaries,Life on Earth - 11. The Hunters and the Hunted,"BBC, iPlayer, TV, Life on Earth, 11. The Hunte...",The eternal duel between the hunters and the h...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,54 mins,3259,27 Mar 1979,The eternal duel between the hunters and the h...,David Attenborough explores the eternal duel b...,David Attenborough explores the eternal duel b...
2775,Documentaries,Life and Birth - Series 1: Episode 4,"BBC, iPlayer, TV, Life and Birth, Series 1: Ep...","Melissa prepares for the birth of Bertie, who ...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,58 mins,3504,12 May 2020,"Melissa prepares for the birth of Bertie, who ...","Melissa prepares for the birth of Bertie, who ...",Melissa didn’t have her first son until she wa...
2776,Documentaries,The Century of the Self - 1. Happiness Machines,"BBC, iPlayer, TV, The Century of the Self, 1. ...",How Sigmund Freuds American nephew invented th...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3511,8pm 17 Mar 2002,How Sigmund Freud's American nephew invented t...,The relationship between the father of psychoa...,The story of the relationship between Sigmund ...
2777,Documentaries,28 Up: Millennium Generation - 14 Up: Episode 1,"BBC, iPlayer, TV, 28 Up: Millennium Generation...",Occasional series charting the progress of a g...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3543,7pm 16 Sep 2007,Occasional series charting the progress of a g...,Occasional documentary series charting the pro...,Occasional documentary series charting the pro...


In [4]:
file_path = '/content/drive/MyDrive/PPM/BBC/history.pkl'
with open(file_path, 'rb') as file:
    bbc_history = pickle.load(file)
bbc_history

,category,title,tags,description,image,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large
0,From the Archives,"The Iraq War - 3. It's Hell, Mr President","BBC, iPlayer, TV, The Iraq War, 3. Its Hell, M...",The final episode tells how the war ended and ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3551,9pm 12 Jun 2013,The final episode tells how the war ended and ...,The biggest players of the war in Iraq reveal ...,The last episode in this three-part series bri...
1,From the Archives,"Silent Witness - Series 9: 3. Choices, Part 1","BBC, iPlayer, TV, Silent Witness, Series 9: 3....",A turf war claims lives on the streets of Lond...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3567,9pm 1 Aug 2005,A turf war claims lives on the streets of Lond...,The team is called to a drive-by shooting outs...,The team is called to a drive-by shooting outs...
2,From the Archives,Not Going Out - The True Meaning of Christmas,"BBC, iPlayer, TV, Not Going Out, The True Mean...",Can Lee and Lucy agree on the true meaning of ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,39 mins,2320,24 Dec 2017,Can Lee and Lucy agree on the true meaning of ...,Sitcom. Can Lee and Lucy agree on the true mea...,Can Lee and Lucy agree on the true meaning of ...
3,From the Archives,Micro Live - Series 3: 17/10/1986,"BBC, iPlayer, TV, Micro Live, Series 3: 17/10/...",Sir Clive Sinclair and Alan Sugar assess chang...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,30 mins,1783,17 Oct 1986,Sir Clive Sinclair and Alan Sugar assess chang...,Micro Live looks at recent changes in the home...,The home computer industry has recently seen i...
4,From the Archives,"BBC Proms - 2021: Last Night of the Proms, Part 2","BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,87 mins,5247,9pm 11 Sep 2021,Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...
...,...,...,...,...,...,...,...,...,...,...,...
4306,From the Archives,Gossip Girl (2007-12) - Series 3: 21. Ex-Husba...,"BBC, iPlayer, TV, Gossip Girl (2007-12), Serie...","When things get tense between Rufus and Will, ...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,40 mins,2419,2009,"When things get tense between Rufus and Will, ...","When things get tense between Rufus and Will, ...","When things get tense between Rufus and Will, ..."
4307,From the Archives,The Century of the Self - 1. Happiness Machines,"BBC, iPlayer, TV, The Century of the Self, 1. ...",How Sigmund Freuds American nephew invented th...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3511,8pm 17 Mar 2002,How Sigmund Freud's American nephew invented t...,The relationship between the father of psychoa...,The story of the relationship between Sigmund ...
4308,From the Archives,Heroes - Series 2: 6. The Line,"BBC, iPlayer, TV, Heroes, Series 2: 6. The Line","Peter goes on a search, as West convinces Clai...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,40 mins,2404,29 May 2008,"Peter goes on a search, as West convinces Clai...",Drama series in which people deal with their n...,Drama series in which people all over the worl...
4309,From the Archives,Merlin - Series 4: 3. The Wicked Day,"BBC, iPlayer, TV, Merlin, Series 4: 3. The Wic...",A carnival to celebrate Prince Arthurs birthda...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,44 mins,2634,8:15pm 15 Oct 2011,A carnival to celebrate Prince Arthur's birthd...,Fantasy drama series. A carnival to celebrate ...,The mighty citadel is buzzing in anticipation ...


In [5]:
file_path = '/content/drive/MyDrive/PPM/BBC/science-and-nature.pkl'
with open(file_path, 'rb') as file:
    bbc_science = pickle.load(file)
bbc_science

,category,title,tags,description,image,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large
0,Science & Nature,The Infinite Monkey Cage - Lucy Beaumont's Clo...,"BBC, iPlayer, TV, The Infinite Monkey Cage, Lu...",Brian Cox & Robin Ince send comedian Lucy Beau...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,12 mins,737,12 Feb 2020,Brian Cox & Robin Ince send comedian Lucy Beau...,Brian Cox and Robin Ince send comedian Lucy Be...,Brian Cox and Robin Ince send writer and comed...
1,Science & Nature,Micro Live - Series 3: 17/10/1986,"BBC, iPlayer, TV, Micro Live, Series 3: 17/10/...",Sir Clive Sinclair and Alan Sugar assess chang...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,30 mins,1783,17 Oct 1986,Sir Clive Sinclair and Alan Sugar assess chang...,Micro Live looks at recent changes in the home...,The home computer industry has recently seen i...
2,Science & Nature,Winterwatch - Series 10: Episode 7,"BBC, iPlayer, TV, Winterwatch, Series 10: Epis...",Megan McCubbin investigates a rather surprisin...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3541,8pm 27 Jan 2022,Megan McCubbin investigates a rather surprisin...,Megan McCubbin is investigating a rather surpr...,Chris Packham and Michaela Strachan bring us t...
3,Science & Nature,"Seven Worlds, One Planet - Series 1: 5. Europe","BBC, iPlayer, TV, Seven Worlds, One Planet, Se...",A continent transformed by mankind where anima...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,58 mins,3484,24 Nov 2019,A continent transformed by mankind where anima...,"Europe - a continent transformed by mankind, w...","Europe, a crowded continent transformed by man..."
4,Science & Nature,Climate Change: Countdown to COP26 - Series 1:...,"BBC, iPlayer, TV, Climate Change: Countdown to...",A special series of programmes helping us make...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,16 mins,965,25 Sep 2021,A special series of programmes helping us make...,"In the run-up to COP26, the BBC's corresponden...","In the run-up to COP26, the BBC's corresponden..."
...,...,...,...,...,...,...,...,...,...,...,...
732,Science & Nature,The Wild Gardener - Series 1: Episode 2,"BBC, iPlayer, TV, The Wild Gardener, Series 1:...",Colin Stafford-Johnson turns his childhood gar...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3540,8pm 26 Nov 2021,Colin Stafford-Johnson turns his childhood gar...,Wildlife cameraman Colin Stafford-Johnson retu...,Wildlife cameraman Colin Stafford-Johnson retu...
733,Science & Nature,The Century of the Self - 1. Happiness Machines,"BBC, iPlayer, TV, The Century of the Self, 1. ...",How Sigmund Freuds American nephew invented th...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3511,8pm 17 Mar 2002,How Sigmund Freud's American nephew invented t...,The relationship between the father of psychoa...,The story of the relationship between Sigmund ...
734,Science & Nature,Hugh's Wild West - Series 1: Episode 10,"BBC, iPlayer, TV, Hughs Wild West, Series 1: E...",Hugh visits an ancient hill farm with dozens o...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,58 mins,3505,10 Mar 2018,Hugh visits an ancient hill farm with dozens o...,Hugh Fearnley-Whittingstall discovers how the ...,Nature lover Hugh Fearnley-Whittingstall conti...
735,Science & Nature,Life on Earth - 11. The Hunters and the Hunted,"BBC, iPlayer, TV, Life on Earth, 11. The Hunte...",The eternal duel between the hunters and the h...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,54 mins,3259,27 Mar 1979,The eternal duel between the hunters and the h...,David Attenborough explores the eternal duel b...,David Attenborough explores the eternal duel b...


In [6]:
file_path = '/content/drive/MyDrive/PPM/BBC/arts.pkl'
with open(file_path, 'rb') as file:
    bbc_arts = pickle.load(file)
bbc_arts

,category,title,tags,description,image,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large
0,Arts,Civilisation - 10. The Smile of Reason,"BBC, iPlayer, TV, Civilisation, 10. The Smile ...",A look at the beginnings of revolutionary poli...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,49 mins,2939,8:15pm 27 Apr 1969,A look at the beginnings of revolutionary poli...,Kenneth Clark looks at the beginnings of revol...,Kenneth Clark looks at the beginnings of revol...
1,Arts,Inside Culture - Series 4: 1. Why We Laugh,"BBC, iPlayer, TV, Inside Culture, Series 4: 1....",Mary Beard asks why we laugh and explores what...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,29 mins,1759,7:30pm 24 Sep 2021,Mary Beard asks why we laugh and explores what...,Mary Beard asks why we laugh and explores what...,Mary Beard asks why we laugh and explores what...
2,Arts,Arena - Agatha Christie - Unfinished Portrait,"BBC, iPlayer, TV, Arena, Agatha Christie - Unf...",Profile celebrating the centenary of the autho...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,61 mins,3642,20 Sep 1990,Profile celebrating the centenary of the autho...,Profile celebrating the centenary of the famou...,Profile celebrating the centenary of the famou...
3,Arts,Fake or Fortune? - Series 9: 4. A King’s Last ...,"BBC, iPlayer, TV, Fake or Fortune?, Series 9: ...",Could a picture bought online for £50 be a mis...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3515,9pm 18 Aug 2021,Could a picture bought online for £50 be a mis...,Anglesey farmer Huw Lewis bought a small paint...,The team head to the beautiful island of Angle...
4,Arts,Andy Warhol's America - Series 1: 2. The Ameri...,"BBC, iPlayer, TV, Andy Warhols America, Series...",Charting Warhols response to the major histori...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3568,6 Jan 2022,Charting Warhol's response to the major histor...,Charting Andy Warhol’s response to major event...,No data found
...,...,...,...,...,...,...,...,...,...,...,...
494,Arts,Inside Cinema - Shorts: 88. Olivia Colman,"BBC, iPlayer, TV, Inside Cinema, Shorts: 88. O...","From sitcom star to screen royalty, Anna Smith...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,8 mins,468,9 Feb 2022,"From sitcom star to screen royalty, Anna Smith...",From sitcom star to Oscar-winning screen royal...,No data found
495,Arts,New Creatives - To My Younger Self,"BBC, iPlayer, TV, New Creatives, To My Younger...",A young man recounts his traumatic experiences...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,4 mins,248,11 Jun 2021,A young man recounts his traumatic experiences...,"A short, creative drama drawing on first hand ...","A short, creative drama drawing on first hand ..."
496,Arts,The Joy of Painting - Series 5: 10. Storm on t...,"BBC, iPlayer, TV, The Joy of Painting, Series ...",Stand on a beach with Bob Ross and admire the ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,26 mins,1533,7:30pm 19 Oct 2021,Stand on a beach with Bob Ross and admire the ...,"Visit this gusty, sandy beach with Bob Ross an...",No data found
497,Arts,Britain's Most Fragile Treasure,"BBC, iPlayer, TV, Britains Most Fragile Treasure","York Minsters Great East Window, the UKs large...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3538,9pm 12 Oct 2011,"York Minster's Great East Window, the UK's lar...",Dr Janina Ramirez unlocks the secrets of a cen...,Historian Dr Janina Ramirez unlocks the secret...


### 3- Data Preparation:

The datasets from different categories are concatenated into a single DataFrame.
A 'content' feature is created by combining 'category', 'title', and 'synopsis_large' into a single text attribute for each show.

In [8]:
bbc_df = pd.concat([bbc_documentries, bbc_history, bbc_science, bbc_arts], ignore_index=True)
bbc_df

,category,title,tags,description,image,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large
0,Documentaries,Fight for First: Excel Esports - Series 1: 1. ...,"BBC, iPlayer, TV, Fight for First: Excel Espor...",Excel Esports owner Kieran is looking for a fr...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,40 mins,2373,10 Jan 2021,Excel Esports owner Kieran is looking for a fr...,Sports documentary series following British un...,Excel is an esports team founded by brothers K...
1,Documentaries,Lady Boss: The Jackie Collins Story,"BBC, iPlayer, TV, Lady Boss: The Jackie Collin...",An immersive journey through the trailblazing ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,91 mins,5467,15 Oct 2021,An immersive journey through the trailblazing ...,An immersive journey through the trailblazing ...,An immersive journey through the trailblazing ...
2,Documentaries,"The Iraq War - 3. It's Hell, Mr President","BBC, iPlayer, TV, The Iraq War, 3. Its Hell, M...",The final episode tells how the war ended and ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3551,9pm 12 Jun 2013,The final episode tells how the war ended and ...,The biggest players of the war in Iraq reveal ...,The last episode in this three-part series bri...
3,Documentaries,The 80s - Music’s Greatest Decade? - Series 1:...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3534,8:55pm 23 Oct 2021,Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...
4,Documentaries,Sacred Rivers with Simon Reeve - 1. The Nile,"BBC, iPlayer, TV, Sacred Rivers with Simon Ree...",Simon Reeve travels from source to sea along t...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3547,9pm 5 Oct 2014,Simon Reeve travels from source to sea along t...,Simon Reeve explores the influence three river...,Adventurer Simon Reeve travels from source to ...
...,...,...,...,...,...,...,...,...,...,...,...
8321,Arts,Inside Cinema - Shorts: 88. Olivia Colman,"BBC, iPlayer, TV, Inside Cinema, Shorts: 88. O...","From sitcom star to screen royalty, Anna Smith...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,8 mins,468,9 Feb 2022,"From sitcom star to screen royalty, Anna Smith...",From sitcom star to Oscar-winning screen royal...,No data found
8322,Arts,New Creatives - To My Younger Self,"BBC, iPlayer, TV, New Creatives, To My Younger...",A young man recounts his traumatic experiences...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,4 mins,248,11 Jun 2021,A young man recounts his traumatic experiences...,"A short, creative drama drawing on first hand ...","A short, creative drama drawing on first hand ..."
8323,Arts,The Joy of Painting - Series 5: 10. Storm on t...,"BBC, iPlayer, TV, The Joy of Painting, Series ...",Stand on a beach with Bob Ross and admire the ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,26 mins,1533,7:30pm 19 Oct 2021,Stand on a beach with Bob Ross and admire the ...,"Visit this gusty, sandy beach with Bob Ross an...",No data found
8324,Arts,Britain's Most Fragile Treasure,"BBC, iPlayer, TV, Britains Most Fragile Treasure","York Minsters Great East Window, the UKs large...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3538,9pm 12 Oct 2011,"York Minster's Great East Window, the UK's lar...",Dr Janina Ramirez unlocks the secrets of a cen...,Historian Dr Janina Ramirez unlocks the secret...


In [9]:
bbc_df['content'] = bbc_df['category'] + " " + bbc_df['title'] + " " + bbc_df['synopsis_large']

### 3-User Data Simulation:

A synthetic user dataset is created to simulate user viewing history, indicating whether a user has previously seen a show.

In [3]:
#Here we create a random set of users to use them to test the  model
# Number of fake users
# num_users = 50

# # Number of documents
# num_bbc = len(bbc_df)

# #Generate random seen/non-seen shows (1 or 0)
# np.random.seed(43)
# seen_data = np.random.choice([0, 1], size=(num_users, num_bbc), p=[0.7, 0.3])

# # Create the DataFrame with shows as rows and users as columns
# user_ids = ['User_{}'.format(i) for i in range(num_users)]
# show_titles = bbc_df['title']

# # Note: Transpose the seen_data to make users columns and titles rows
# user_df = pd.DataFrame(seen_data.T, index=show_titles, columns=user_ids)

# # Save the DataFrame to a csv file
# csv_file_path = '/content/drive/MyDrive/PPM/BBC/users_bbc.csv'
# user_df.to_csv(csv_file_path)

# user_df.transpose()
user_df = pd.read_csv("/content/drive/MyDrive/PPM/BBC/users_bbc.csv")
user_df

,title,User_0,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,...,User_40,User_41,User_42,User_43,User_44,User_45,User_46,User_47,User_48,User_49
0,Fight for First: Excel Esports - Series 1: 1. ...,0,0,0,1,0,0,0,0,0,...,1,0,1,1,0,0,0,0,1,0
1,Lady Boss: The Jackie Collins Story,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2,"The Iraq War - 3. It's Hell, Mr President",0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
3,The 80s - Music’s Greatest Decade? - Series 1:...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Sacred Rivers with Simon Reeve - 1. The Nile,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,Inside Cinema - Shorts: 88. Olivia Colman,0,0,0,1,1,1,1,0,1,...,0,0,1,0,0,1,0,0,0,0
8322,New Creatives - To My Younger Self,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,1,1,0,1,0
8323,The Joy of Painting - Series 5: 10. Storm on t...,0,1,0,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
8324,Britain's Most Fragile Treasure,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# #bbc_df = bbc_df.reset_index()
# print(bbc_df.columns)

# print(user_df.columns)

In [12]:
#user_df.reset_index()

,title,User_0,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,...,User_40,User_41,User_42,User_43,User_44,User_45,User_46,User_47,User_48,User_49
0,Fight for First: Excel Esports - Series 1: 1. ...,0,0,0,1,0,0,0,0,0,...,1,0,1,1,0,0,0,0,1,0
1,Lady Boss: The Jackie Collins Story,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2,"The Iraq War - 3. It's Hell, Mr President",0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
3,The 80s - Music’s Greatest Decade? - Series 1:...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Sacred Rivers with Simon Reeve - 1. The Nile,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,Inside Cinema - Shorts: 88. Olivia Colman,0,0,0,1,1,1,1,0,1,...,0,0,1,0,0,1,0,0,0,0
8322,New Creatives - To My Younger Self,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,1,1,0,1,0
8323,The Joy of Painting - Series 5: 10. Storm on t...,0,1,0,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
8324,Britain's Most Fragile Treasure,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
#bbc_df.reset_index()

,index,category,title,tags,description,image,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large,content
0,0,Documentaries,Fight for First: Excel Esports - Series 1: 1. ...,"BBC, iPlayer, TV, Fight for First: Excel Espor...",Excel Esports owner Kieran is looking for a fr...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,40 mins,2373,10 Jan 2021,Excel Esports owner Kieran is looking for a fr...,Sports documentary series following British un...,Excel is an esports team founded by brothers K...,Documentaries Fight for First: Excel Esports -...
1,1,Documentaries,Lady Boss: The Jackie Collins Story,"BBC, iPlayer, TV, Lady Boss: The Jackie Collin...",An immersive journey through the trailblazing ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,91 mins,5467,15 Oct 2021,An immersive journey through the trailblazing ...,An immersive journey through the trailblazing ...,An immersive journey through the trailblazing ...,Documentaries Lady Boss: The Jackie Collins St...
2,2,Documentaries,"The Iraq War - 3. It's Hell, Mr President","BBC, iPlayer, TV, The Iraq War, 3. Its Hell, M...",The final episode tells how the war ended and ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3551,9pm 12 Jun 2013,The final episode tells how the war ended and ...,The biggest players of the war in Iraq reveal ...,The last episode in this three-part series bri...,"Documentaries The Iraq War - 3. It's Hell, Mr ..."
3,3,Documentaries,The 80s - Music’s Greatest Decade? - Series 1:...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3534,8:55pm 23 Oct 2021,Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Documentaries The 80s - Music’s Greatest Decad...
4,4,Documentaries,Sacred Rivers with Simon Reeve - 1. The Nile,"BBC, iPlayer, TV, Sacred Rivers with Simon Ree...",Simon Reeve travels from source to sea along t...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3547,9pm 5 Oct 2014,Simon Reeve travels from source to sea along t...,Simon Reeve explores the influence three river...,Adventurer Simon Reeve travels from source to ...,Documentaries Sacred Rivers with Simon Reeve -...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,8321,Arts,Inside Cinema - Shorts: 88. Olivia Colman,"BBC, iPlayer, TV, Inside Cinema, Shorts: 88. O...","From sitcom star to screen royalty, Anna Smith...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,8 mins,468,9 Feb 2022,"From sitcom star to screen royalty, Anna Smith...",From sitcom star to Oscar-winning screen royal...,No data found,Arts Inside Cinema - Shorts: 88. Olivia Colman...
8322,8322,Arts,New Creatives - To My Younger Self,"BBC, iPlayer, TV, New Creatives, To My Younger...",A young man recounts his traumatic experiences...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,4 mins,248,11 Jun 2021,A young man recounts his traumatic experiences...,"A short, creative drama drawing on first hand ...","A short, creative drama drawing on first hand ...",Arts New Creatives - To My Younger Self A shor...
8323,8323,Arts,The Joy of Painting - Series 5: 10. Storm on t...,"BBC, iPlayer, TV, The Joy of Painting, Series ...",Stand on a beach with Bob Ross and admire the ...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,26 mins,1533,7:30pm 19 Oct 2021,Stand on a beach with Bob Ross and admire the ...,"Visit this gusty, sandy beach with Bob Ross an...",No data found,Arts The Joy of Painting - Series 5: 10. Storm...
8324,8324,Arts,Britain's Most Fragile Treasure,"BBC, iPlayer, TV, Britains Most Fragile Treasure","York Minsters Great East Window, the UKs large...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,59 mins,3538,9pm 12 Oct 2011,"York Minster's Great East Window, the UK's lar...",Dr Janina Ramirez unlocks the secrets of a cen...,Historian Dr 

In [37]:
# Create a content profile for each item (documentary) by combining text fields
#bbc_df['content'] = bbc_df['category'] + " " + bbc_df['title'] + " " + bbc_df['synopsis_large']
#user_df.drop(columns=['views'], axis=1, inplace=True)
# user_df["views"] = user_df.iloc[:, 1:51].sum(axis=1)

# user_df["popularity"] = user_df['views']/50
# user_df
# user_df
# bbc_df
#bbc_df = bbc_df.merge(user_df, on='title', how='left')
# bbc_df.drop(columns=['views_x', 'views_y'], axis=1, inplace=True)
# bbc_df
# bbc_df_1 = pd.merge(bbc_df[['title', 'category', 'content']], user_df[['title', 'views', 'popularity']], on='title')
# bbc_df_1
#bbc_df['views'] = user_df['views']
#bbc_df['popularity'] = user_df['popularity'].values

# bbc_df_final = bbc_df[['category', 'title', 'content', 'popularity']]
# bbc_df_final

In [4]:
# file_path2 = '/content/drive/MyDrive/PPM/BBC/bbc_df_final.csv'
# bbc_df_final.to_csv(file_path2)
bbc_df_final = pd.read_csv('/content/drive/MyDrive/PPM/BBC/bbc_df_final.csv')
bbc_df_final

,Unnamed: 0,category,title,content,popularity
0,0,Documentaries,Fight for First: Excel Esports - Series 1: 1. ...,Documentaries Fight for First: Excel Esports -...,0.52
1,1,Documentaries,Lady Boss: The Jackie Collins Story,Documentaries Lady Boss: The Jackie Collins St...,0.44
2,2,Documentaries,"The Iraq War - 3. It's Hell, Mr President","Documentaries The Iraq War - 3. It's Hell, Mr ...",0.64
3,3,Documentaries,The 80s - Music’s Greatest Decade? - Series 1:...,Documentaries The 80s - Music’s Greatest Decad...,0.40
4,4,Documentaries,Sacred Rivers with Simon Reeve - 1. The Nile,Documentaries Sacred Rivers with Simon Reeve -...,0.32
...,...,...,...,...,...
8321,8321,Arts,Inside Cinema - Shorts: 88. Olivia Colman,Arts Inside Cinema - Shorts: 88. Olivia Colman...,0.52
8322,8322,Arts,New Creatives - To My Younger Self,Arts New Creatives - To My Younger Self A shor...,0.56
8323,8323,Arts,The Joy of Painting - Series 5: 10. Storm on t...,Arts The Joy of Painting - Series 5: 10. Storm...,0.68
8324,8324,Arts,Britain's Most Fragile Treasure,Arts Britain's Most Fragile Treasure Historian...,0.32


### 5- Recommendation System:
####A-Serendipitous Recommender System

A function (get_recommendation_serendipity) is defined to generate show recommendations based on serendipity, considering both relevance (using cosine similarity) and unexpectedness (based on show popularity).


In [5]:

def get_recommendation_serendipity(shows_df, user_df, user_id, unexpectedness_weight, relevance_weight):
    '''
    S = R * U
    Relevance: all seen shows and those unseen shows that have high similarities (I used cosine similarity here)
    Unexpectedness: all shows with popularity less than average (we calculate the popularity score based on the number of users who watched the show for simplicity)
    Serendipity: product of both scores after weighting them
    '''
    #first we alculate popularity score based on the number of users who watched each show
    popularity_scores = shows_df['popularity']
    average_popularity = popularity_scores.mean()

    # get the user's seen shows based on user_df
    user_seen = user_df[user_id] == 1
    #print(user_seen)
    # Vectorize the content using TF-IDF
    tfidf = TfidfVectorizer(strip_accents='unicode',  lowercase=True, stop_words='english')
    tfidf_matrix = tfidf.fit_transform(shows_df['content'])
    #normalize tfidf_matrix
    tfidf_matrix = normalize(tfidf_matrix)
    #print(tfidf_matrix)
    # measure cosine similarity matrix for the shows' content
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    #print(cosine_sim)

    # Get the indices of the shows the user has seen

    #seen_indices = user_seen.index[user_seen].tolist()
    seen_indices = np.where(user_seen)[0]
    #print(seen_indices)
    #print(seen_indices)
    #seen_indices_array = np.array(seen_indices, dtype=int)


    # measure average similarity score for unseen shows
    relevance_scores = []
    for i in range(cosine_sim.shape[0]):
        if not user_seen.iloc[i]:
            #only consider similarities with shows the user has seen
            similarity_scores = cosine_sim[i, seen_indices]
            relevance_scores.append(np.mean(similarity_scores))
        else:
            relevance_scores.append(0)

    #finally, the relevance score
    relevance_scores = np.array(relevance_scores)

    # measure unexpectedness for each show based on its popularity (if they are less than average, we consider them unexpected)
    unexpectedness_scores = np.where(popularity_scores < average_popularity, (1- popularity_scores)/average_popularity, 0)

    # apply weights to the relevance and unexpectedness scores
    weighted_relevance = relevance_scores * relevance_weight
    weighted_unexpectedness = unexpectedness_scores * unexpectedness_weight

    #measure serendipity scores as the product of weighted relevance and weighted unexpectedness
    serendipity_scores = weighted_relevance * weighted_unexpectedness

    #generate list of unseen shows along with their serendipity scores
    unseen_shows_serendipity = [(shows_df.iloc[i]['title'], score)
                                for i, score in enumerate(serendipity_scores) if not user_seen[i]]

    #sort them based on their serendipity scores
    sorted_unseen_shows = sorted(unseen_shows_serendipity, key=lambda x: x[1], reverse=True)

    #get the top 5 recommendations
    top_recommendations = sorted_unseen_shows[:5]
    recommendations_df = pd.DataFrame(top_recommendations, columns=['Title', 'Serendipity Score'])

    return recommendations_df




####B- Baseline Recommender System
Another function (recommend_similar_shows) is defined for a baseline recommender system that recommends shows based on similarity to previously watched shows.

In [8]:
#baseline Recommender system


def recommend_similar_shows(shows_df, user_df, user_id):
    tfidf = TfidfVectorizer(strip_accents='unicode', lowercase=True, stop_words='english')
    tfidf_matrix = tfidf.fit_transform(shows_df['content'])

    #cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    user_seen_indices = user_df.index[user_df[user_id] == 1].tolist()

    user_profile_vector = tfidf_matrix[user_seen_indices].mean(axis=0)
    user_profile_vector = np.squeeze(np.asarray(user_profile_vector))
    user_profile_vector = user_profile_vector.reshape(1, -1)
    cosine_similarities = linear_kernel(user_profile_vector, tfidf_matrix).flatten()
    #cosine_similarities = linear_kernel(user_profile_vector, tfidf_matrix).flatten()

    shows_df['similarity_score'] = cosine_similarities

    unseen_df = shows_df[~bbc_df_final.index.isin(user_seen_indices)]

    sorted_by_similarity = unseen_df.sort_values('similarity_score', ascending=False)
    top_recommendations = sorted_by_similarity[:5]

    recommendations_df = pd.DataFrame(top_recommendations, columns=['title', 'similarity_score'])


    return recommendations_df

### 6- Model Application and Results:


In [7]:
recommendations = get_recommendation_serendipity(bbc_df_final, user_df, 'User_1', 0.9, 0.1)
print(recommendations)

                                     Title  Serendipity Score
0     Little Britain - Series 1: Episode 7           0.004011
1           Top Gear - Series 1: Episode 4           0.003858
2  Call the Midwife - Series 10: Episode 1           0.003590
3      Waterloo Road - Series 1: Episode 7           0.003394
4     Waterloo Road - Series 4: Episode 19           0.003359


In [9]:
recommendations= recommend_similar_shows(bbc_df_final, user_df, 'User_1')
print(recommendations)

                               title  similarity_score
4631  Top Gear - Series 1: Episode 8          0.042011
2967  Top Gear - Series 1: Episode 4          0.042011
2905  Top Gear - Series 1: Episode 3          0.042011
5186  Top Gear - Series 1: Episode 5          0.042011
6587  Top Gear - Series 1: Episode 7          0.042011
